# Similar Regions examples

## Initialize

In [5]:
import pandas as pd
import numpy as np
import logging
import nest_asyncio
from api.client.samples.similar_regions_Frechet.sr import SimilarRegion

nest_asyncio.apply()

sim = SimilarRegion()

Example of preloading the data for a particular part of the world (this step is optional, you can skip ahead to Examples below).

In [6]:
%%time
# If search_region = 0 ("World"), this takes many hours the first time)
# sim._logger.setLevel(logging.INFO)
sim._load_region_info(root_region_ids=[1065])
sim._load_data()

CPU times: user 491 ms, sys: 8.11 ms, total: 499 ms
Wall time: 498 ms


# Examples

Find districts in Ethiopia similar to Napa county, California, USA.

In [7]:
print("Similar to Napa:")
for i in sim.similar_to(136969, number_of_regions=10, requested_level=5, compare_to=1065):
    print(i)

Similar to Napa:
{'#': 0, 'id': 136969, 'name': 'Napa', 'dist': 3.650024149988857e-08, 'parent': ('', '', '', '')}
{'#': 1, 'id': 115015, 'name': 'Eastern', 'dist': 2.179871841826362, 'parent': (10928, 'Tigray', 1065, 'Ethiopia')}
{'#': 2, 'id': 142811, 'name': 'Guji', 'dist': 2.2823705100922163, 'parent': (10925, 'Oromia', 1065, 'Ethiopia')}
{'#': 3, 'id': 114991, 'name': 'Liben', 'dist': 2.3559261928001587, 'parent': (10926, 'Somali', 1065, 'Ethiopia')}
{'#': 4, 'id': 142824, 'name': 'South East', 'dist': 2.4452399031747416, 'parent': (10928, 'Tigray', 1065, 'Ethiopia')}
{'#': 5, 'id': 114979, 'name': 'Borena', 'dist': 2.4907974208975663, 'parent': (10925, 'Oromia', 1065, 'Ethiopia')}
{'#': 6, 'id': 142818, 'name': 'Afder', 'dist': 2.5459836750927067, 'parent': (10926, 'Somali', 1065, 'Ethiopia')}
{'#': 7, 'id': 115011, 'name': 'South Omo', 'dist': 2.5553233985662738, 'parent': (10927, 'Southern Nations, Nationalities and Peoples', 1065, 'Ethiopia')}
{'#': 8, 'id': 115014, 'name': 'C

Find provinces in Europe similar to the state of Iowa, USA.

In [8]:
# Find provinces similar to Iowa in Europe (with detailed distance report)
print("Similar to Iowa:")
for i in sim.similar_to(13066, number_of_regions=10, requested_level=4,detailed_distance=True, compare_to=14):
    print(i)

Property cation_exchange_30cm is missing 14/9418 regions
  0%|          | 0/14 [00:00<?, ?it/s]

Similar to Iowa:


100%|██████████| 14/14 [00:00<00:00, 42.78it/s]
/home/nemo/src/api-client/api/client/samples/similar_regions_Frechet/sr.py:384: RuntimeWarning: invalid value encountered in true_divide
  valid_data = valid_data/data_counters # division by zero where we do not have data
Property clay_30cm is missing 14/9418 regions
100%|██████████| 14/14 [00:00<00:00, 54.42it/s]
Property land_surface_temperature is missing 11/9418 regions
100%|██████████| 11/11 [00:06<00:00,  1.64it/s]
Property organic_carbon_content_fine_earth_30cm is missing 14/9418 regions
100%|██████████| 14/14 [00:00<00:00, 51.64it/s]
Property ph_h2o_30cm is missing 14/9418 regions
100%|██████████| 14/14 [00:00<00:00, 59.25it/s]
Property rainfall is missing 9/9418 regions
100%|██████████| 9/9 [00:13<00:00,  1.45s/it]
Property sand_30cm is missing 14/9418 regions
100%|██████████| 14/14 [00:00<00:00, 41.15it/s]
Property silt_30cm is missing 14/9418 regions
100%|██████████| 14/14 [00:00<00:00, 50.06it/s]
Property soil_moisture is miss

{'#': 0, 'id': 13066, 'name': 'Iowa', 'dist': {'total': 7.146345858741878e-08, 'covar': 7.146345858741878e-08, 'cation_exchange_30cm': 0.0, 'clay_30cm': 0.0, 'land_surface_temperature': 0.0, 'organic_carbon_content_fine_earth_30cm': 0.0, 'ph_h2o_30cm': 0.0, 'rainfall': 0.0, 'sand_30cm': 0.0, 'silt_30cm': 0.0, 'soil_moisture': 0.0, 'soil_water_capacity_100cm': 0.0}, 'parent': ('', '', '', '')}
{'#': 1, 'id': 11711, 'name': 'Donduseni', 'dist': {'total': 1.0428215244587287, 'covar': 0.45939844703256627, 'cation_exchange_30cm': 0.013829020865511854, 'clay_30cm': 0.007930746874838412, 'land_surface_temperature': 0.060937054224847964, 'organic_carbon_content_fine_earth_30cm': 0.24718140286487522, 'ph_h2o_30cm': 0.6013490863036823, 'rainfall': 0.392014270271549, 'sand_30cm': 0.14219195559385867, 'silt_30cm': 0.1810348910280144, 'soil_moisture': 0.49302690832238905, 'soil_water_capacity_100cm': 0.00024828346454341954}, 'parent': (1134, 'Moldova', '', '')}
{'#': 2, 'id': 11714, 'name': 'Edinet